# =============================================================================
# PASO 1: CONFIGURACIÓN DEL ENTORNO E IMPORTACIÓN DE LIBRERÍAS
# =============================================================================
# Importamos las librerías fundamentales para el análisis y modelado.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Librerías de Scikit-Learn para preprocesamiento y modelado
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Modelos de clasificación
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Métricas de evaluación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("Librerías importadas correctamente.")



In [ ]:
# =============================================================================
# PASO 2: CARGA DE DATOS
# =============================================================================
# Para este ejemplo, usaremos el dataset del Titanic, que está disponible en Seaborn.
# Este dataset es ideal para practicar porque contiene valores nulos y tipos de datos mixtos.
print("\nCargando el dataset del Titanic...")
df = sns.load_dataset('titanic')
print("Dataset cargado. Aquí están las primeras 5 filas:")
print(df.head())



In [ ]:
# =============================================================================
# PASO 3: ANÁLISIS EXPLORATORIO DE DATOS (EDA)
# =============================================================================
# El EDA es fundamental para entender la estructura, la calidad y las relaciones en los datos.

print("\n--- Iniciando Análisis Exploratorio de Datos (EDA) ---")

# 3.1. Información General del Dataset
print("\nInformación general del DataFrame (df.info()):")
df.info()

# 3.2. Resumen Estadístico de Variables Numéricas
print("\nResumen estadístico de variables numéricas (df.describe()):")
print(df.describe())

# 3.3. Análisis de Valores Nulos
print("\nConteo de valores nulos por columna:")
print(df.isnull().sum())

# Visualización de valores nulos
plt.figure(figsize=(10, 6))
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')
plt.title('Mapa de Calor de Valores Nulos')
plt.show()

# 3.4. Análisis Univariado (Análisis de variables individuales)
print("\nRealizando análisis univariado...")

# Supervivencia (nuestra variable objetivo)
sns.countplot(x='survived', data=df)
plt.title('Distribución de Supervivencia (0 = No, 1 = Sí)')
plt.show()

# Clases de pasajeros
sns.countplot(x='pclass', data=df)
plt.title('Distribución de Clases de Pasajeros')
plt.show()

# Género
sns.countplot(x='sex', data=df)
plt.title('Distribución por Género')
plt.show()

# Distribución de Edad
sns.histplot(df['age'].dropna(), kde=True, bins=30)
plt.title('Distribución de Edades')
plt.show()

# 3.5. Análisis Bivariado (Relación entre dos variables)
print("\nRealizando análisis bivariado...")

# Supervivencia vs. Clase
sns.countplot(x='survived', hue='pclass', data=df)
plt.title('Supervivencia por Clase de Pasajero')
plt.show()

# Supervivencia vs. Género
sns.countplot(x='survived', hue='sex', data=df)
plt.title('Supervivencia por Género')
plt.show()

# Supervivencia vs. Edad
g = sns.FacetGrid(df, col='survived', height=5)
g.map(sns.histplot, 'age', bins=20, kde=False)
plt.suptitle('Distribución de Edad por Supervivencia', y=1.02)
plt.show()

# 3.6. Matriz de Correlación para variables numéricas
print("\nCalculando matriz de correlación...")
# Creamos una copia para el heatmap, seleccionando solo columnas numéricas
numeric_cols = df.select_dtypes(include=np.number)
correlation_matrix = numeric_cols.corr()

plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Matriz de Correlación de Variables Numéricas')
plt.show()



In [ ]:
# =============================================================================
# PASO 4: PREPROCESAMIENTO DE DATOS E INGENIERÍA DE CARACTERÍSTICAS
# =============================================================================
print("\n--- Iniciando Preprocesamiento de Datos ---")

# Eliminamos columnas que no aportan información útil o son difíciles de tratar
# 'deck' tiene demasiados nulos, 'embark_town' es redundante con 'embarked', etc.
df_processed = df.drop(['deck', 'embark_town', 'alive', 'who', 'adult_male', 'class'], axis=1)

# Separamos la variable objetivo (y) de las predictoras (X)
X = df_processed.drop('survived', axis=1)
y = df_processed['survived']

# Identificamos columnas numéricas y categóricas
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object', 'category', 'bool']).columns

print(f"Columnas numéricas: {list(numerical_features)}")
print(f"Columnas categóricas: {list(categorical_features)}")

# Creamos pipelines de preprocesamiento para cada tipo de dato
# Para datos numéricos: imputamos la media y luego escalamos.
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# Para datos categóricos: imputamos la moda y luego aplicamos One-Hot Encoding.
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Usamos ColumnTransformer para aplicar los pipelines a las columnas correctas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ])

# Dividimos los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"\nDatos divididos: {len(X_train)} para entrenamiento, {len(X_test)} para prueba.")



In [ ]:
# =============================================================================
# PASO 5: ENTRENAMIENTO Y EVALUACIÓN DE MODELOS
# =============================================================================
print("\n--- Entrenando y Evaluando Modelos ---")

# Definimos los modelos que vamos a probar
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "Support Vector Machine": SVC(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42)
}

# Iteramos sobre cada modelo para entrenarlo y evaluarlo
results = {}
for name, model in models.items():
    # Creamos un pipeline completo que incluye preprocesamiento y el modelo
    model_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                     ('classifier', model)])

    # Entrenamos el pipeline
    model_pipeline.fit(X_train, y_train)

    # Realizamos predicciones
    y_pred = model_pipeline.predict(X_test)

    # Calculamos la precisión
    accuracy = accuracy_score(y_test, y_pred)
    results[name] = accuracy

    # Imprimimos los resultados
    print(f"\n--- Resultados para: {name} ---")
    print(f"Accuracy: {accuracy:.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    # Matriz de confusión
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusión - {name}')
    plt.xlabel('Predicho')
    plt.ylabel('Verdadero')
    plt.show()



In [ ]:
# =============================================================================
# PASO 6: COMPARACIÓN DE MODELOS
# =============================================================================
print("\n--- Comparación Final de Modelos ---")
results_df = pd.DataFrame(list(results.items()), columns=['Model', 'Accuracy']).sort_values(by='Accuracy', ascending=False)
print(results_df)

plt.figure(figsize=(10, 6))
sns.barplot(x='Accuracy', y='Model', data=results_df)
plt.title('Comparación de la Precisión de los Modelos')
plt.xlim(0.7, 0.9)
plt.show()

print("\nAnálisis finalizado.")
print("El modelo con mejor rendimiento inicial es el Gradient Boosting.")
print("El siguiente paso podría ser la optimización de hiperparámetros para este modelo.")